In [1]:
import torch

In [38]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

gold = torch.tensor([0, 1, 2])

pred = torch.tensor([[1000, -1000, -1000],
                    [-1000, 1000, -1000],
                    [-1000, -1000, 1000]]) /1001

loss = criterion(pred, gold)

print(loss)


tensor(0.3732)


In [43]:
# Add as a new code cell

import torch
import torch.nn.functional as F

vocab_size = 5
epsilon    = 0.1
true_label = 0  # pick any class, e.g. 0

# build the smoothed “true” distribution q
q = torch.full((vocab_size,), epsilon / (vocab_size - 1))
q[true_label] = 1.0 - epsilon

# 1) theoretical min CE = H(q) = -∑ q_i log q_i
min_ce = -torch.sum(q * torch.log(q))
print(f"Theoretical minimum CE (ε={epsilon}): {min_ce:.6f}")
perplexity = torch.exp(min_ce)
print(f"Perplexity (ε={epsilon}): {perplexity:.6f}")
# 2) simulate a model that outputs exactly q
#    so logits = log q (logits → softmax → q# filepath: /Users/lucasfragara/Thesis/min_ce.ipynb
# Add as a new code cell


Theoretical minimum CE (ε=0.1): 0.463712
Perplexity (ε=0.1): 1.589966


In [45]:
# Add as another new code cell

import torch
import torch.nn as nn

# reuse vocab_size, epsilon, true_label, q from above or redefine them

# 3) simulate a model that outputs exactly q
#    so logits = log q  ⇒ softmax(logits) = q
logits = torch.log(q).unsqueeze(0)        # shape (1, vocab_size)
targets = torch.tensor([true_label])      # shape (1,)

# 4) compute CE with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=epsilon)
loss = criterion(logits, targets)

print(f"CrossEntropyLoss w/ smoothing={epsilon}: {loss:.6f}")
# should match the theoretical min CE printed above
print(torch.exp(loss))

CrossEntropyLoss w/ smoothing=0.1: 0.392042
tensor(1.4800)


In [58]:
import json

with open('text.json', 'r') as f:
    text = json.loads(f.read())

from data.CFG_parsers import CFGParser
from data.grammars import GRAMMAR_CFG3b as grammar
parser = CFGParser(grammar)


text = text['data']

In [61]:
for el in text: 
    full_original = el['prompt'] + el['generated']
    print(parser.is_valid(full_original))

False
False
False
False
False
